**1.Обучите любую модель классификации на датасете IRIS до применения PCA и после него. Сравните качество классификации по отложенной выборке.**

In [382]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [383]:
iris = datasets.load_iris()
X = iris.data
y = iris.target
y.shape, X.shape

((150,), (150, 4))

In [384]:
def standard_scale(x):
    res = (x - x.mean(axis=0)) / x.std(axis=0)
    return res

In [385]:
X = standard_scale(X)

In [386]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=26, test_size=0.3)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((105, 4), (105,), (45, 4), (45,))

In [387]:
model = RandomForestClassifier(random_state=26)

In [388]:
%%time
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

CPU times: total: 219 ms
Wall time: 299 ms


In [389]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       0.94      1.00      0.97        15
           2       1.00      0.94      0.97        16

    accuracy                           0.98        45
   macro avg       0.98      0.98      0.98        45
weighted avg       0.98      0.98      0.98        45



In [390]:
# Получаем матрицу ковариации
covariance_matrix = X_train.T @ X_train

#Получаем собственные значения и собственные вектора
eig_values, eig_vectors = np.linalg.eig(covariance_matrix)

#Возьмём первые 2 из собственных векторов, это будут наши веса
W = np.hstack([eig_vectors.T[i].reshape(4,1) for i in range(2)])

#Снизим размерность матриц с помощью полученных весов
X_train_reduced = X_train.dot(W)
X_test_reduced = X_test.dot(W)
X_train_reduced.shape, X_test_reduced.shape

((105, 2), (45, 2))

In [391]:
%%time
model.fit(X_train_reduced, y_train)
y_pred_reduced = model.predict(X_test_reduced)

CPU times: total: 203 ms
Wall time: 265 ms


In [392]:
print(classification_report(y_test, y_pred_reduced))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       0.82      0.93      0.87        15
           2       0.93      0.81      0.87        16

    accuracy                           0.91        45
   macro avg       0.92      0.92      0.91        45
weighted avg       0.92      0.91      0.91        45



In [393]:
#Сравним со встроенным РСА

from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X_train_reduced = pca.fit_transform(X_train)
X_test_reduced = pca.transform(X_test)
X_train_reduced.shape, X_test_reduced.shape

((105, 2), (45, 2))

In [394]:
%%time
model.fit(X_train_reduced, y_train)
y_pred_reduced = model.predict(X_test_reduced)

CPU times: total: 281 ms
Wall time: 371 ms


In [395]:
print(classification_report(y_test, y_pred_reduced))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       0.82      0.93      0.87        15
           2       0.93      0.81      0.87        16

    accuracy                           0.91        45
   macro avg       0.92      0.92      0.91        45
weighted avg       0.92      0.91      0.91        45



**Метрики закономерно упали, всё-таки 4 признака заменили всего 1. Однако упали не критично, минимальное значение из всех-всех 0.81 (было 0.94), в то время как время обучения и предсказания удалось снизить за счёт сокращения кол-ва признаков. F1 мера осталась на достойном уровне**

### Задание 2
**Напишите свою реализацию метода главных компонент посредством сингулярного разложения с использованием функции numpy.linalg.svd().**

In [396]:
# Функция для получения матрицы весов

def svd_weights(X, num=2):
    #Находим сингулярное разложение матрицы
    u, s, vh = np.linalg.svd(X, full_matrices=True)
    # vh - собственные векторы, как раз необходимые нам для получения весов
    
    #Формируем матрицу весов из стобцов матрицы V
    W_svd = np.hstack([vh[i].reshape(X.shape[1],1) for i in range(num)])
    return W_svd


# Функция для снижения размерности (получения новой матрицы объекты-признаки)
def svd_reduce(X, weights):
    X_reduced = X.dot(weights)
    return X_reduced

In [397]:
#Получаем веса на обучающей выборке
weights = svd_weights(X_train, num = 2) 
#Сокращаем размерности матриц обучающей и тестовой
X_train_svd_reduced = svd_reduce(X_train, weights)
X_test_svd_reduced = svd_reduce(X_test, weights)
X_train_svd_reduced.shape, X_test_svd_reduced.shape

((105, 2), (45, 2))

In [398]:
%%time
model.fit(X_train_svd_reduced, y_train)
y_pred_svd_reduced = model.predict(X_test_svd_reduced)

CPU times: total: 672 ms
Wall time: 302 ms


In [399]:
print(classification_report(y_test, y_pred_svd_reduced))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       0.82      0.93      0.87        15
           2       0.93      0.81      0.87        16

    accuracy                           0.91        45
   macro avg       0.92      0.92      0.91        45
weighted avg       0.92      0.91      0.91        45



**Все метрики совпали с предыдущими матрицами**